In [63]:
# import itertools as it
# from multiprocessing import Pool, cpu_count
# from collections import defaultdict
import re
import numpy as np
from functools import reduce
from operator import mul
from bisect import insort
with open('input.txt', 'r') as file:
    input = file.read()

test_input2 = """\
#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
"""
test_input1= """\
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
"""

### Part 1
Ok. So path finding. Running towards my closest path finding algo rn
Probably use the complex number coords as well bc it worked out yesterday

In [81]:
# ref:https://medium.com/@nicholas.w.swift/easy-a-star-pathfinding-7e6689c7f7b2
#                          ^ wow! like santa!
class Node():

    def __init__(self, parent=None, position=None, direction=None):
        self.parent = parent
        self.position = position
        self.dir = direction
        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position and self.dir == other.dir
    
    def __lt__(self, other):
        return self.f < other.f
    
    def __str__(self):
        return str((int(self.position.real),int(self.position.imag)))
    
    def __hash__(self):
        return hash((self.position,self.dir))

class ReindeerMaze():
    def __init__(self,txt):
        self.grid = txt.splitlines()
        # self.dirs =  {'>':1j,'v':1,'^':-1,'<':-1j}
        self.dirs=[1j,1,-1,-1j]
        self.obs= set()
        self.cost = 0
        # self.end = 0
        # self.start = Node(None,position)
        print(self.grid)
        
        for x in range(len(self.grid)):
            for y in range(len(self.grid[0])):
                # print(x,'',y)
                c=self.grid[x][y]
                if c == '#':
                    self.obs.add(x+y*1j)
                if c == 'S':
                    self.start=Node(None,x+y*1j,1j) # facing east
                if c == 'E':
                    self.end=Node(None,x+y*1j)
        print(f"Start node: {self.start.position}, {self.start.dir}")
        print(f"End node: {self.end.position}")

    def char_at_pos(self,pos):
        return self.grid[int(pos.real)][int(pos.imag)]
    
    def a_star_search(self):
        open,closed = [],{}
        insort(open,self.start)
        total_paths=[]

        # counter = 2000

        while(open):
            # insort(open, item)

            # get current (min(f))
            current = open.pop(0)

            # if found return path
            if current.position == self.end.position:
                print("found")
                path = []
                c = current
                self.cost = c.g
                while c is not None:
                    path.append(c.position)
                    c = c.parent
                total_paths.append(path[::-1]) # Return reversed path
                continue
            
            # add to closed
            if current.position not in closed:
                closed[current.position] = []
            closed[current.position].append(current)


            # define and loop through children
            children = []
            for dir in self.dirs:
                new_pos = current.position + dir
                if new_pos not in self.obs and 0<=new_pos.real<len(self.grid) and 0<=new_pos.imag<len(self.grid[0]):
                    children.append(Node(current,new_pos,dir))
           
            for child in children:
                # print(child)
                if child in closed: continue
                
                child.g = current.g + 1
                # here's the important part: if rotate, +1000
                if child.dir == current.dir*1j or child.dir == current.dir*-1j: 
                    # print("turned=", current,child)
                    child.g += 1000
                child.h = abs(self.end.position.real-child.position.real)+abs(self.end.position.imag-child.position.imag)
                child.f = child.g+child.h
                
                # if child node doesnt perform better than any node in open
                # just skip to next node
                # for open_node in open:
                #     if child.position == open_node.position and child.g > open_node.g:
                #         continue

                if child.position in closed:
                    if any( closed_node.g <= child.g for closed_node in closed[child.position]):
                        continue
                # in other words, if it is
                insort(open,child)

            print(f"Current: {current.position}, g={current.g}, h={current.h}, f={current.f}")
            print("Children:")
            for child in children:
                print(f"  {child.position}, g={child.g}, h={child.h}, f={child.f}")
            print("Open list:", [str(node) for node in open])
            # counter-=1
        # Return all found paths
        if total_paths:
            print(f"Found {len(total_paths)} paths to the goal.")
            return total_paths
        else:
            print("No paths found.")
            return None

res = ReindeerMaze(test_input1)
paths = res.a_star_search()

for path in paths:
    print(path)
    print(len(path))

print("part1:", res.cost)


['###############', '#.......#....E#', '#.#.###.#.###.#', '#.....#.#...#.#', '#.###.#####.#.#', '#.#.#.......#.#', '#.#.#####.###.#', '#...........#.#', '###.#.#####.#.#', '#...#.....#.#.#', '#.#.#.###.#.#.#', '#.....#...#.#.#', '#.###.#.#.#.#.#', '#S..#.....#...#', '###############']
Start node: (13+1j), 1j
End node: (1+13j)
Current: (13+1j), g=0, h=0, f=0
Children:
  (13+2j), g=1, h=23.0, f=24.0
  (12+1j), g=1001, h=23.0, f=1024.0
Open list: ['(13, 2)', '(12, 1)']
Current: (13+2j), g=1, h=23.0, f=24.0
Children:
  (13+3j), g=2, h=22.0, f=24.0
  (13+1j), g=2, h=24.0, f=26.0
Open list: ['(13, 3)', '(12, 1)']
Current: (13+3j), g=2, h=22.0, f=24.0
Children:
  (13+2j), g=3, h=23.0, f=26.0
Open list: ['(12, 1)']
Current: (12+1j), g=1001, h=23.0, f=1024.0
Children:
  (13+1j), g=1002, h=24.0, f=1026.0
  (11+1j), g=1002, h=22.0, f=1024.0
Open list: ['(11, 1)']
Current: (11+1j), g=1002, h=22.0, f=1024.0
Children:
  (11+2j), g=2003, h=21.0, f=2024.0
  (12+1j), g=1003, h=23.0, f=1026.0
  (10+1j),

### Part 2
Ok so I already kept track of one best path with this implementation. but what if i wanted multiple?
- keep searching
- add found path to a ~~list~~ set?

In [101]:
# the working version with HyperNeutrino's logics added
class ReindeerMaze:
    def __init__(self,txt):
        self.grid = txt.splitlines()
        # self.dirs =  {'>':1j,'v':1,'^':-1,'<':-1j}
        self.dirs=[1j,1,-1,-1j]
        self.obs= set()
        self.cost = 0
        self.end = None
        self.start = None # Node(None,position)
        print(self.grid)
        
        for x in range(len(self.grid)):
            for y in range(len(self.grid[0])):
                # print(x,'',y)
                c=self.grid[x][y]
                if c == '#':
                    self.obs.add(x+y*1j)
                if c == 'S':
                    self.start=Node(None,x+y*1j,1j) # facing east
                if c == 'E':
                    self.end=Node(None,x+y*1j)
        print(f"Start node: {self.start.position}, {self.start.dir}")
        print(f"End node: {self.end.position}")

    def char_at_pos(self,pos):
        return self.grid[int(pos.real)][int(pos.imag)]
    
    def a_star_search(self):
        open = []
        lowest_cost = {} # node, g
        insort(open,self.start)
        total_paths=[]
        best_cost = float("inf")

        while(open):
            # get current (min(f))
            current = open.pop(0)
            
            # register in lowest_cost (aka "closed")
            if current.g > lowest_cost.get(current, float("inf")): continue
            lowest_cost[current] = current.g

            # if found return path
            if current.position == self.end.position:
                # run out of optimal paths
                if current.g > best_cost: break 
                best_cost = current.g
                # build path
                path = []
                c = current
                self.cost = c.g
                while c is not None:
                    path.append(c.position)
                    c = c.parent
                total_paths.append(path[::-1]) # Return reversed path
                continue 


            # define and loop through children
            children = []
            for dir in self.dirs:
                new_pos = current.position + dir
                if new_pos not in self.obs and 0<=new_pos.real<len(self.grid) and 0<=new_pos.imag<len(self.grid[0]):
                    # difference: first calculate the children
                    child = Node(current,new_pos,dir)
                    child.g = current.g + 1
                    # here's the important part: if rotate, +1000
                    if child.dir == current.dir*1j or child.dir == current.dir*-1j: 
                        child.g += 1000
                    child.h = abs(self.end.position.real-child.position.real)+abs(self.end.position.imag-child.position.imag)
                    child.f = child.g #+child.h
                    children.append(child)

           
            for child in children:
                if current.g > lowest_cost.get(child,float("inf")):
                    continue
                insort(open,child)

            # print(f"Current: {current.position}, g={current.g}, h={current.h}, f={current.f}")
            # print("Children:")
            # for child in children:
            #     print(f"  {child.position}, g={child.g}, h={child.h}, f={child.f}")
            # print("Open list:", [str(node) for node in open])
            # counter-=1
        # Return all found paths
        if total_paths:
            print(f"Found {len(total_paths)} paths to the goal.")
            return total_paths
        else:
            print("No paths found.")
            return None

    def find_seats(self):
        s = set()
        paths = res.a_star_search()
        for path in paths:
            # print(path)
            s.update(path)
        return s


res = ReindeerMaze(input)
print("part2:", len(res.find_seats()))


['#############################################################################################################################################', '#.....#.............#.#.......#.......#.................#.......#...#.......#.........#.....#.......#.......#...............#..............E#', '#.###.#.###########.#.#.#####.#.#.###.#######.#########.#.#####.#.#.#.#####.#.#####.###.#.#.#.#######.#.###.#.###.#####.###.#.###.#.#.#.#.###', '#...#...#.....#.....#.#.#...................#.#.......#.................#.#.#...#.#.....#.#...#.......#.#.#.#...#.....#...#.#.#.............#', '###.#######.#.#.#####.#.#.#.###.###.#######.#.#####.#.#######.###.#.###.#.#.#.#.#.#######.#####.#######.#.#.#.#.#########.#.#.#.#######.###.#', '#...#...#...#.#.....#.....#.....#...#.#.....#.....#.#.#...#...#...#.......#...#.#...#...#.....#.#.......#.#.#.#...........#.#.#.#...#.......#', '#.###.#.#.#.#.#####.#.###########.###.#.#########.#.###.#.#.#.#.#############.#.###.#.#.#####.#.#.#######.#.#.###############.#.

### Takeaways
- The trick to part 2 was to figure out that the **closed set** simply served the purpose to **filtering out calculated nodes**. In a separate use case, it need to filter out something else instead -- aka, nodes with costs highter than recorded. My problem was not being familiar enough with the algorithm enough to see the logics without hint.
- Interestingly enough, for the second part I've taken out of the heuristic and it'd ran faster. Note to self to know which pathfinding to pick. Having implemented the backtracking beforehand was a good thing to keep in mind though.